In [2]:
from brancharchitect.split import PartitionSet, Partition
from brancharchitect.jumping_taxa.lattice.lattice_construction import (
    check_non_subsumption_with_residual,
    check_atomic_inclusion,
)

In [3]:
def check_independence_for_partitions(
    left_covets, right_covets, left_residual, right_residual, lookup=None
):
    """
    Test check_independence_conditions with custom partition sets.

    Args:
        left_covets: List of tuples representing partitions for left side
        right_covets: List of tuples representing partitions for right side
        left_residual: List of tuples representing left residual (b-a)
        right_residual: List of tuples representing right residual (a-b)
        lookup: Optional dictionary mapping strings to indices

    Returns:
        tuple: (left_non_subsumption, right_non_subsumption, left_atomic_inclusion, right_atomic_inclusion)
    """
    # Create lookup if not provided
    if lookup is None:
        # Create a simple numeric lookup
        elements = set()
        for partition in left_covets + right_covets + left_residual + right_residual:
            elements.update(partition)
        lookup = {str(i): i for i in elements}

    # Create the partition sets
    left_partition_set = PartitionSet(
        {Partition(p, lookup) for p in left_covets}, lookup
    )
    right_partition_set = PartitionSet(
        {Partition(p, lookup) for p in right_covets}, lookup
    )
    left_residual_set = PartitionSet(
        {Partition(p, lookup) for p in left_residual}, lookup
    )
    right_residual_set = PartitionSet(
        {Partition(p, lookup) for p in right_residual}, lookup
    )

    # Create the dictionaries
    left = {"covet_left": left_partition_set, "b-a": left_residual_set}

    right = {"covet_right": right_partition_set, "a-b": right_residual_set}

    # Extract partition sets
    left_arm = left.get("covet_left", PartitionSet())
    right_arm = right.get("covet_right", PartitionSet())

    # Condition 1: Left partition is not fully contained
    # and has a non-empty right-side residual
    left_non_subsumption = check_non_subsumption_with_residual(
        primary_set=left_arm,
        comparison_set=right_arm,
        residual=left.get("b-a", PartitionSet()),
    )

    # Condition 2: Right partition is not fully contained
    # and has a non-empty left-side residual
    right_non_subsumption = check_non_subsumption_with_residual(
        primary_set=right_arm,
        comparison_set=left_arm,
        residual=right.get("a-b", PartitionSet()),
    )

    # Condition 3: Left atomic inclusion
    # (single element set not subset of a larger set)
    left_atomic_inclusion = check_atomic_inclusion(
        primary_set=left_arm, comparison_set=right_arm
    )

    # Condition 4: Right atomic inclusion
    # (single element set not subset of a larger set)
    right_atomic_inclusion = check_atomic_inclusion(
        primary_set=right_arm, comparison_set=left_arm
    )

    return (
        left_non_subsumption,
        right_non_subsumption,
        left_atomic_inclusion,
        right_atomic_inclusion,
    )


def test_independence_cases():
    """Test multiple independence cases."""

    # Define lookup for named elements
    lookup = {
        "X1": 0,
        "X2": 1,
        "A1": 2,
        "A2": 3,
        "A3": 4,
        "A4": 5,
        "B1": 6,
        "B2": 7,
        "B3": 8,
        "B4": 9,
    }

    lookup2 = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6, "H": 7}
    lookup_4274 = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4}
    lookup_5911 = {"O1": 0, "O2": 1, "3": 2, "4": 3, "5": 4, "6": 5}
    lookup_7604 = {"A": 0, "B": 1, "C": 2, "E": 3, "D": 4}
    # Test cases
    test_cases = [
        # Case 1: test_tree_moving_updwards.json
        {
            "name": "Original example",
            "left_covets": [
                PartitionSet({(2,), (3,), (4,)}, lookup),
                PartitionSet({(0, 1)}, lookup),
            ],
            "right_covets": [
                PartitionSet({(2,)}, lookup),
                PartitionSet({(3,), (4,), (0, 1)}, lookup),
            ],
            "expected": (True, False, False, False),
        },
        # Case 2: basic_1_taxon_partial.json
        {
            "name": "Example with B, X, A groups",
            "left_covets": [
                PartitionSet({(1,), (2,), (3,), (4, 5)}, lookup),
                PartitionSet({(6, 7, 8, 9)}, lookup),
            ],
            "right_covets": [
                PartitionSet({(1,)}, lookup),
                PartitionSet({(2,), (3,), (6, 7, 8, 9)}, lookup),
            ],
            "expected": (False, True, False, False),
        },
        # Case 3: Simple atomic inclusion example
        # small-test-one-leave-crossing-moving.json
        {
            "name": "Atomic inclusion example",
            "left_covets": [
                PartitionSet({(0,), (1,)}, lookup2),
                PartitionSet({(2,), (3,), (4,)}, lookup2),
            ],
            "right_covets": [
                PartitionSet({(1,), (3,)}, lookup2),
                PartitionSet({(0,), (2,), (4,)}, lookup2),
            ],
            "expected": (True, True, True, False),
        },
        # Case 4: basic_1_taxon_none.json
        {
            "name": "Atomic inclusion example",
            "left_covets": [
                PartitionSet({(2, 3, 4, 5), (1,)}, lookup2),
                PartitionSet({(6, 7, 8, 9)}, lookup2),
            ],
            "right_covets": [
                PartitionSet({(2, 3, 4, 5)}, lookup2),
                PartitionSet({(6, 7, 8, 9)}, lookup2),
                PartitionSet({(1,)}, lookup2),
            ],
            "expected": (True, True, True, False),
        },
        # Case 4: heiko_4_test_tree.json
        {
            "name": "Atomic inclusion example",
            "left_covets": [
                PartitionSet({(3,), (4, 5, 6), (7,)}, lookup2),
                PartitionSet({(0, 1), (2,)}, lookup2),
            ],
            "right_covets": [
                PartitionSet({(0, 1), (2,), (4, 5, 6)}, lookup2),
                PartitionSet({(3), (7)}, lookup2),
            ],
            "expected": (True, True, True, False),
        },
        {
            "name": "Initial",
            "left_covets": [
                PartitionSet({(0,), (1,)}, lookup_4274),
                PartitionSet({(2,), (3,), (4,)}, lookup_4274),
            ],
            "right_covets": [
                PartitionSet({(1,), (3,)}, lookup_4274),
                PartitionSet({(0,), (2,), (4,)}, lookup_4274),
            ],
            "expected": (True, True, True, False),  # Update with expected results
        },
        {
            "name": "Initial",
            "left_covets": [
                PartitionSet({(2,), (3,)}, lookup_5911),
                PartitionSet({(4,), (5,)}, lookup_5911),
            ],
            "right_covets": [
                PartitionSet({(2,)}, lookup_5911),
                PartitionSet({(3,)}, lookup_5911),
                PartitionSet({(4,)}, lookup_5911),
                PartitionSet({(5,)}, lookup_5911),
            ],
            "expected": (True, True, True, False),  # Update with expected results
        },
        {
            "name": "After",
            "left_covets": [
                PartitionSet({(2,)}, lookup_7604),
                PartitionSet({(4,)}, lookup_7604),
            ],
            "right_covets": [
                PartitionSet({(2,)}, lookup_7604),
                PartitionSet({(4,)}, lookup_7604),
            ],
            "expected": (True, True, True, False),  # Update with expected results
        },
    ]

    # Run tests
    for i, case in enumerate(test_cases):
        print(f"\nTest case {i+1}: {case['name']}")
        results = check_independence_for_partitions(
            case["left_covets"],
            case["right_covets"],
            case["left_residual"],
            case["right_residual"],
            lookup,
        )
        print(f"Expected: {case['expected']}")
        print(f"Actual:   {results}")
        assert (
            results == case["expected"]
        ), f"Case {i+1} failed: expected {case['expected']}, got {results}"
        print("✓ Passed")

# Call the test function
test_independence_cases()


Test case 1: Original example


KeyError: 'left_residual'